In [1]:
# with open('data/sentences.txt', 'w', encoding='utf-8') as f:
#     for sentence in sentences['sentence']:
#         f.write('{}\n'.format(sentence))

In [84]:
import pandas as pd
import glob
import os
import glob
import subprocess
import csv

In [3]:
pd.set_option('display.max_colwidth', -1)

In [78]:
datasets = []
for genre in ['academic', 'fiction', 'newspapers', 'conversation']:
    dataset = pd.read_csv('data/train/{}/words.csv'.format(genre),
                          encoding='ISO--8859-1', na_filter=False)
    display(dataset.head())
    datasets.append(dataset)

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,25070,HER,she,DPS,,,1267,a6u-fragment02,a6u,academic
1,25071,DRESS,dress,NN1,,,1267,a6u-fragment02,a6u,academic
2,25072,HANGS,hang,VVZ,,,1267,a6u-fragment02,a6u,academic
3,25073,HERE',here',NP0,,,1267,a6u-fragment02,a6u,academic
4,25074,DE-FROCKING,de-frock,VVG,mrw,met,1267,a6u-fragment02,a6u,academic


,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,38263,The,the,AT0,,,1947,ab9-fragment03,ab9,fiction
1,38264,boy,boy,NN1,,,1947,ab9-fragment03,ab9,fiction
2,38265,and,and,CJC,,,1947,ab9-fragment03,ab9,fiction
3,38266,the,the,AT0,,,1947,ab9-fragment03,ab9,fiction
4,38267,dog,dog,NN1,,,1947,ab9-fragment03,ab9,fiction


,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,0,Latest,late,AJS,,,0,a1e-fragment01,a1e,newspapers
1,1,corporate,corporate,AJ0,,,0,a1e-fragment01,a1e,newspapers
2,2,unbundler,unbundler,NN1,,,0,a1e-fragment01,a1e,newspapers
3,3,reveals,reveal,VVZ,mrw,met,0,a1e-fragment01,a1e,newspapers
4,4,laid-back,laid-back,AJ0,,,0,a1e-fragment01,a1e,newspapers


,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,156256,Ours,ours,PNP,,,8721,kb7-fragment10,kb7,conversation
1,156257,is,be,VBZ,,,8721,kb7-fragment10,kb7,conversation
2,156258,actually,actually,AV0,,,8721,kb7-fragment10,kb7,conversation
3,156259,a,a,AT0,,,8721,kb7-fragment10,kb7,conversation
4,156260,new,new,AJ0,,,8721,kb7-fragment10,kb7,conversation


In [79]:
dataset = pd.concat(datasets)
len(dataset)

164303

In [80]:
all_sentences = pd.read_csv('data/raw/sentences.csv', encoding='ISO-8859-1')
display(len(all_sentences))
display(all_sentences.sample(5))
all_sentences.tail(3)

16183

,sentence_id,paragraph_id,sentence
6266,6266,1933,"See, for example, Fig. 5.3"
2874,2874,942,Unemployment was 80 per cent on some streets.
14369,14369,-1,You can use your cloth daddy
5631,5631,1747,"It was a dramatic and impractical outfit of a kind I only expected to see on the male models who posed in the more outlandish fashion magazines that our rich clients brought aboard, yet Jesse Isambard Sweetman managed to wear the elaborate style with an elegant insouciance."
7702,7702,2299,Nevertheless a few qualitative conclusions may be drawn without doing any further mathematics.


,sentence_id,paragraph_id,sentence
16180,16180,-1,Some else's sitting at their desk?
16181,16181,-1,Well not you know cleaning so I do n't know what kind of work they do.
16182,16182,-1,Oh well if you 're here that 's all right


In [81]:
# Sentences the training set contains.
sentences = all_sentences.merge(dataset)[['sentence_id', 'paragraph_id', 'sentence']].drop_duplicates()
display(sentences['sentence_id'].nunique())
sentences.sample(5)

13134

,sentence_id,paragraph_id,sentence
149369,13261,-1,Nobody mentioned that they were puppets themselves.
40504,2565,847,"In its place would be a sentencing discretion, which judges would use, in many cases, to mark the relative heinousness of the murder by a determinate prison sentence."
154648,14452,-1,"You 're gon na help me, you 're gon na stay here I 'm afraid"
151522,13670,-1,Do you want a straw mum?
163506,16035,-1,Dear me!


## Build paragraphs

In [15]:
paragraph_sentences = sentences[sentences['paragraph_id'] != -1]
display(len(paragraph_sentences))  # Number of sentences that belong to paragraphs.
paragraph_sentences.sample(5)

6561

,sentence_id,paragraph_id,sentence
95548,6241,1928,"The reader is required to follow a series of steps which might, for example, introduce a change in the performance of a complex system."
115819,8059,2390,There is a potty for emergencies.
47922,3027,988,But Stewart snaps: I do n't want to be disappointed so I 'm not going to ask for their help.
13559,774,257,It does n't matter.
110072,7481,2245," A white man has blood, red blood also."


In [33]:
paragraphs = paragraph_sentences.groupby('paragraph_id')['sentence'].agg(list).apply(lambda x: ' '.join(x))
display(len(paragraphs))
paragraphs.head()

2067

paragraph_id
3    IT SEEMS that Roland Franklin, the latest unbundler to appear in the UK, has made a fatal error in the preparation of his £697m break-up bid for stationery and packaging group DRG. He has not properly investigated the target's dining facilities.                                                                                                                   
4    The 63-year-old head of Pembridge Investments, through which the bid is being mounted says, rule number one in this business is: the more luxurious the luncheon rooms at headquarters, the more inefficient the business. If he had taken his own rule seriously, he would have found out that DRG has a very modest self-service canteen at its Bristol head office.
5    There are other things he has, on his own admission, not fully investigated, like the value of the DRG properties, or which part of the DRG business he would keep after the break up. When the bid was launched last week, Mr Franklin faced some critici

## Save paragraphs

This saves all the paragraphs we created from our sentences.
```python
!rm data/train/paragraphs/*
for i, paragraph in paragraphs.iteritems():
    file_name = "data/train/paragraphs/paragraph_{}.txt".format(i)
    with open(file_name, 'w', encoding='utf-8') as f:
        print('Writing {}'.format(file_name))
        f.write(paragraph)
```

Next, run the following.
```bash
python scripts/paragraph_chain.py
```

## Explore Longest Chains

In [150]:
longest_chains = pd.read_csv('data/train/longest_chains.csv')  # Paragraphs with longest chains != ''

longest_chain_in_paragraphs = longest_chains.merge(paragraphs, left_on='paragraph_id', right_index=True)

# We should have a row for each paragraph with a non-empty longest_chain.
display(len(paragraphs))
display(sum(longest_chains['paragraph_id'].isin(paragraphs.index)))
display(len(longest_chain_in_paragraphs['paragraph_id'].drop_duplicates()))
display(len(longest_chain_in_paragraphs))  
display(len(longest_chains))

longest_chain_in_paragraphs.head()

2067

1519

1519

1519

1519

,paragraph_id,longest_chain,sentence
0,10,Loading the Index... Loading the Text... Loading the Categories... holding businesses matter consequence genius concept company company,What he has learned from Goldsmith( the only genius I have ever come across) is that the holding company is not the most important unit of corporate organisation.  The concept of the company does not interest me- it 's the businesses that are important. It does not matter whether or not DRG makes sellotape or Basildon Bond it 's of no consequence to anybody if somebody else makes them.
1,1000,Loading the Index... Loading the Text... Loading the Categories... ran found crop unqualified lost obliged rest money cattle,"Nor could the vaunted irrigation scheme be described as an unqualified success. The money ran out before the scheme's completion, and the villagers found themselves obliged to borrow the rest. They begged Rytasha for reimbursement, but none was forthcoming, and for the want of a few hundred pounds the village lost its crop for the year.  Now what are we supposed to do? asked one of them.  Are we to sell our cattle or our land?"
2,1001,Loading the Index... Loading the Text... Loading the Categories... spiritual gurus teach form speak language,"The most irritating thing about Rytasha and there were many was that, though she does not speak their language, she presumes to teach them a spurious form of their own religion.  One thing we do n't need in this subcontinent, Professor Rokeya Kabeer observed, is spiritual guidance we have gurus a dime a dozen."
3,1002,Loading the Index... Loading the Text... Loading the Categories... lines white,"They are far too civilised for that sort of thing in Bangladesh, but I kept thinking wistfully of T. S. Eliot's lines in Sweeney Agonistes about a nice little white little missionary stew."
4,1003,Loading the Index... Loading the Text... Loading the Categories... stock market index investors holders market surge offer risen,"AT LEAST last Friday's post-election stock market surge was not a one-day special offer for the professionals only. Prices have remained the FT-SE index has risen another 55 points since then allowing even the most passive of private investors, including unit trust holders, to take advantage of the market."


Looks pretty good!

## Now let's put our code into reuseable and automated functions!

In [243]:
dataset = pd.read_csv('data/train/academic/words.csv'.format(genre),
                      encoding='ISO--8859-1', na_filter=False)

In [244]:
def build_paragraphs(dataset):
    all_sentences = pd.read_csv('data/raw/sentences.csv', encoding='ISO-8859-1')

    # Sentences the given dataset contains.
    sentences = all_sentences.merge(dataset)[['sentence_id', 'paragraph_id', 'sentence']].drop_duplicates()

    paragraph_sentences = sentences[sentences['paragraph_id'] != -1]

    paragraphs = paragraph_sentences.groupby('paragraph_id')['sentence'].agg(list).apply(lambda x: ' '.join(x))
    return paragraphs


def save_paragraphs(paragraphs):
    try:
        files = glob.glob('data/tmp/paragraphs/*')
        for f in files:
            os.remove(f)
    except:
        os.mkdir('data/tmp/paragraphs')
        
    for i, paragraph in paragraphs.iteritems():
        file_name = "data/tmp/paragraphs/paragraph_{}.txt".format(i)
        with open(file_name, 'w', encoding='utf-8') as f:
            print('Writing {}'.format(file_name))
            f.write(paragraph)  


def get_longest_chains(genre):
    # Go to ELKB directory from the parent directory of this file.
#     dir = os.path.dirname(os.path.realpath(__file__))
#     os.chdir(os.path.join(dir, '../ELKB'))

    with open('data/tmp/longest_chains.csv'.format(genre), 'w', encoding='utf-8') as f:
        csv_writer = csv.writer(f, delimiter=',')
        csv_writer.writerow(('paragraph_id', 'longest_chain'))

        for file_name in glob.glob('data/tmp/paragraphs/paragraph*.txt'.format(genre)):
            paragraph_id = os.path.splitext(os.path.basename(file_name))[
                0].split('_')[-1]

            with open(file_name, 'r', encoding='utf-8') as g:
                result = subprocess.run(
                    'java applications/LexicalChain -f ../{}'.format(file_name),
                    cwd='ELKB/',
                    stdout=subprocess.PIPE)

                result = result.stdout.decode('utf-8')

                result = result.strip().split('\n')

                # Does not contain a lexical chain.
                # First 3 lines are descriptives by the script.
                if len(result) < 4:
                    continue

                longest_chain = []
                len_longest_chain = 0
                for r in result[3:]:
                    chain = [word.strip()
                             for word in r.split('[')[0].split(',')]
                    if len(chain) >= len_longest_chain:
                        longest_chain.extend(chain)
                        
                    len_longest_chain = len(chain)
                    
                # longest_chain = result[3]
                # longest_chain = [word.strip() for word in longest_chain.split('[')[0].split(',')]
                longest_chain_str = ' '.join(longest_chain)

                # line = '{},{}'.format(paragraph_id, longest_chain)
                # f.write(line + '\n')
                row = [paragraph_id, longest_chain_str]
#                 print(row)
                print('Writing longest chain for {} with {} words!\n'
                      .format(file_name, len(longest_chain)))
                csv_writer.writerow(row)

def get_lc_feature(dataset):
    longest_chains = pd.read_csv('tmp/longest_chains.csv')  # Paragraphs with longest chains != ''

    longest_chain_in_paragraphs = longest_chains.merge(paragraphs, left_on='paragraph_id', right_index=True)

    words_and_longest_chains = (
        longest_chain_in_paragraphs
        .merge(sentences)
        .merge(dataset[['sentence_id', 'word_id', 'word']])
    )

    words_and_longest_chains['longest_chain_list'] = (
        words_and_longest_chains['longest_chain']
        .apply(lambda x: set(x.split(' ')))
    )

    words_and_longest_chains['is_in_longest_chain'] = [
        word in chain for (word, chain) in zip(
            words_and_longest_chains['word'],
            words_and_longest_chains['longest_chain_list']
        )
    ]

    return words_and_longest_chains[['word_id', 'is_in_longest_chain']]

def add_lc_feature(dataset, genre):
    paragraphs = build_paragraphs(dataset)
    save_paragraphs(paragraphs)
    
    get_longest_chains(genre)
    
    lc_feature = get_lc_feature(dataset)
    
    dataset_with_lc_feature = dataset.merge(lc_feature, how='left')
    dataset_with_lc_feature['is_in_longest_chain'].fillna(False)

    return dataset_with_lc_feature

Great! Now let's try our new model with the added Lexical Chain feature!

In [249]:
dataset = pd.read_csv('data/train/academic/words.csv', encoding='ISO--8859-1', na_filter=False)
display(len(dataset))
dataset.head()

48964

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,25070,HER,she,DPS,,,1267,a6u-fragment02,a6u,academic
1,25071,DRESS,dress,NN1,,,1267,a6u-fragment02,a6u,academic
2,25072,HANGS,hang,VVZ,,,1267,a6u-fragment02,a6u,academic
3,25073,HERE',here',NP0,,,1267,a6u-fragment02,a6u,academic
4,25074,DE-FROCKING,de-frock,VVG,mrw,met,1267,a6u-fragment02,a6u,academic


In [298]:
%run model_functions.ipynb

In [ ]:
genre = 'academic'
train_set = pd.read_csv('data/train/{}/words.csv'.format(genre), encoding='ISO--8859-1', na_filter=False)
X_train, test_fold_ids, vocabulary, y_train = preprocess_data(train_set, want_lc=True)

unigram_classifier = UnigramClassifier(
    verbose=10, random_state=0, C=10, penalty='l1',
    solver='liblinear', class_weight='balanced', max_iter=500,
    vocabulary=vocabulary
)

C = np.logspace(-2,4,20)
penalty = ['l1', 'l2']
hyperparameters = {'C':C, 'penalty':penalty}
clf = GridSearchCV(unigram_classifier, hyperparameters,
                   cv=PredefinedSplit(test_fold_ids),
                   verbose=10, scoring='f1')

clf.fit(X_train, y_train)

test_set = pd.read_csv('data/test/{}/words.csv'.format(genre), encoding='ISO--8859-1', na_filter=False)
X_test, _, _, y_test = preprocess_data(test_set)
preds = clf.predict(X_test)

result = classification_report(y_test, preds, output_dict)
print(result)

-----------Writing data/tmp/paragraphs/paragraph_412.txt
Writing data/tmp/paragraphs/paragraph_413.txt
Writing data/tmp/paragraphs/paragraph_414.txt
Writing data/tmp/paragraphs/paragraph_415.txt
Writing data/tmp/paragraphs/paragraph_416.txt
Writing data/tmp/paragraphs/paragraph_417.txt
Writing data/tmp/paragraphs/paragraph_418.txt
Writing data/tmp/paragraphs/paragraph_419.txt
Writing data/tmp/paragraphs/paragraph_420.txt
Writing data/tmp/paragraphs/paragraph_421.txt
Writing data/tmp/paragraphs/paragraph_422.txt
Writing data/tmp/paragraphs/paragraph_423.txt
Writing data/tmp/paragraphs/paragraph_424.txt
Writing data/tmp/paragraphs/paragraph_425.txt
Writing data/tmp/paragraphs/paragraph_426.txt
Writing data/tmp/paragraphs/paragraph_427.txt
Writing data/tmp/paragraphs/paragraph_840.txt
Writing data/tmp/paragraphs/paragraph_841.txt
Writing data/tmp/paragraphs/paragraph_842.txt
Writing data/tmp/paragraphs/paragraph_843.txt
Writing data/tmp/paragraphs/paragraph_844.txt
Writing data/tmp/paragr

Writing data/tmp/paragraphs/paragraph_1220.txt
Writing data/tmp/paragraphs/paragraph_1221.txt
Writing data/tmp/paragraphs/paragraph_1222.txt
Writing data/tmp/paragraphs/paragraph_1223.txt
Writing data/tmp/paragraphs/paragraph_1224.txt
Writing data/tmp/paragraphs/paragraph_1225.txt
Writing data/tmp/paragraphs/paragraph_1226.txt
Writing data/tmp/paragraphs/paragraph_1227.txt
Writing data/tmp/paragraphs/paragraph_1228.txt
Writing data/tmp/paragraphs/paragraph_1229.txt
Writing data/tmp/paragraphs/paragraph_1230.txt
Writing data/tmp/paragraphs/paragraph_1231.txt
Writing data/tmp/paragraphs/paragraph_1232.txt
Writing data/tmp/paragraphs/paragraph_1233.txt
Writing data/tmp/paragraphs/paragraph_1234.txt
Writing data/tmp/paragraphs/paragraph_1235.txt
Writing data/tmp/paragraphs/paragraph_1236.txt
Writing data/tmp/paragraphs/paragraph_1237.txt
Writing data/tmp/paragraphs/paragraph_1238.txt
Writing data/tmp/paragraphs/paragraph_1239.txt
Writing data/tmp/paragraphs/paragraph_1240.txt
Writing data/

Writing data/tmp/paragraphs/paragraph_2055.txt
Writing data/tmp/paragraphs/paragraph_2056.txt
Writing data/tmp/paragraphs/paragraph_2057.txt
Writing data/tmp/paragraphs/paragraph_2058.txt
Writing data/tmp/paragraphs/paragraph_2059.txt
Writing data/tmp/paragraphs/paragraph_2060.txt
Writing data/tmp/paragraphs/paragraph_2061.txt
Writing data/tmp/paragraphs/paragraph_2062.txt
Writing data/tmp/paragraphs/paragraph_2063.txt
Writing data/tmp/paragraphs/paragraph_2064.txt
Writing data/tmp/paragraphs/paragraph_2065.txt
Writing data/tmp/paragraphs/paragraph_2066.txt
Writing data/tmp/paragraphs/paragraph_2067.txt
Writing data/tmp/paragraphs/paragraph_2068.txt
Writing data/tmp/paragraphs/paragraph_2069.txt
Writing data/tmp/paragraphs/paragraph_2070.txt
Writing data/tmp/paragraphs/paragraph_2071.txt
Writing data/tmp/paragraphs/paragraph_2072.txt
Writing data/tmp/paragraphs/paragraph_2073.txt
Writing data/tmp/paragraphs/paragraph_2074.txt
Writing data/tmp/paragraphs/paragraph_2075.txt
Writing data/